In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In [2]:
case class Message(source: Long, value: (Double, Double), destPosInDistribution: Int)
case class Node(typeOfNode: Int, neighbors: Int, distribution: Array[Double], messages: List[Message], firstDestination: Long)

defined class Message
defined class Node


In [3]:
val fileName = "../data/input.txt"
var lines = sc.textFile(fileName).collect()
var variablesNumber: Long = 0
var factorsNumber: Long = 0
variablesNumber = lines(0).split(" ")(0).toLong
factorsNumber = lines(0).split(" ")(1).toLong
var tempNodes: Array[(Long, Node)] = new Array[(Long, Node)]((variablesNumber+factorsNumber).toInt) // (id, (type, leaf, distribution))
for (a <- 0 to (variablesNumber+factorsNumber - 1).toInt){
    tempNodes(a) = (a.toLong, Node(0,0, null, List[Message](), -1))
}
var tempEdges: Array[Edge[(Int)]] = new Array[Edge[(Int)]](((variablesNumber+factorsNumber-1)*2).toInt)
var cont = 0
for (a <- 1 to lines.size-1){
    if(a%2 == 1){
        val factorNode = variablesNumber + ((a-1)/2).toLong
        var pos = 0
        for (variableNode <- lines(a).split(" ")){
            tempEdges(cont) = Edge(variableNode.toLong, factorNode, pos)
            cont = cont + 1
            tempEdges(cont) = Edge(factorNode, variableNode.toLong, pos)
            cont = cont + 1
            pos = pos + 1
        } 
    }else{
        val distribution:  Array[Double] = lines(a).split(" ").map(_.toDouble)
        val factorNode = (variablesNumber + ((a-1)/2).toLong).toInt
        tempNodes(factorNode) = (factorNode, Node(1, 0, distribution, List[Message](), -1))
    }
}

fileName = ../data/input.txt
lines = Array(5 5, 0 1, 0.25 0.25 0.25 0.25, 1 2 3, 0.16706443914081145 0.01909307875894988 0.18138424821002386 0.1431980906921241 0.02386634844868735 0.15513126491646778 0.07159904534606205 0.2386634844868735, 2, 0.4 0.6, 3 4, 0.20 0.30 0.40 0.10, 3, 0.25 0.75)
variablesNumber = 5
factorsNumber = 5
variablesNumber = 5
factorsNumber = 5
tempNodes = Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@387ecba5,List(),-1)), (6,Node(1,0,[D@16f1200f,List(),-1)), (7,Node(1,0,[D@50f59dc1,List(),-1)), (8,Node(1,0,[D@28fdc538,List(),-1)), (9,Node(1,0,[D@66100061,List(),-1)))


tempEdge...


Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@387ecba5,List(),-1)), (6,Node(1,0,[D@16f1200f,List(),-1)), (7,Node(1,0,[D@50f59dc1,List(),-1)), (8,Node(1,0,[D@28fdc538,List(),-1)), (9,Node(1,0,[D@66100061,List(),-1)))

domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani
domani


In [4]:
val nodes: RDD[(VertexId, Node)] = sc.parallelize(tempNodes)
val edges: RDD[Edge[Int]] = sc.parallelize(tempEdges)
val graph = Graph(nodes, edges)

nodes = ParallelCollectionRDD[2] at parallelize at <console>:39
edges = ParallelCollectionRDD[3] at parallelize at <console>:40
graph = org.apache.spark.graphx.impl.GraphImpl@4917d70a


org.apache.spark.graphx.impl.GraphImpl@4917d70a

In [21]:
val inDegrees: VertexRDD[Int] = graph.inDegrees
//leaves.collect().foreach(println)
val graphWithLeaves = graph.joinVertices(inDegrees){
 case (id, node, neighbors) =>  Node(node.typeOfNode, neighbors, node.distribution, node.messages, node.firstDestination)
}
//graphWithLeaves.vertices.collect().foreach(println)

(4,Node(0,1,null,List(),-1))
(0,Node(0,1,null,List(),-1))
(8,Node(1,2,[D@190386eb,List(),-1))
(1,Node(0,2,null,List(),-1))
(9,Node(1,1,[D@1c9d1142,List(),-1))
(5,Node(1,2,[D@1902c211,List(),-1))
(6,Node(1,3,[D@7bf69645,List(),-1))
(2,Node(0,2,null,List(),-1))
(3,Node(0,3,null,List(),-1))
(7,Node(1,1,[D@1701dbd,List(),-1))


inDegrees = VertexRDDImpl[19] at RDD at VertexRDD.scala:57
graphWithLeaves = org.apache.spark.graphx.impl.GraphImpl@75131bd0


org.apache.spark.graphx.impl.GraphImpl@75131bd0

In [6]:
def getValueFromDistribution(distribution: Array[Double], pos: Array[Int]): Double = {
    var index: Int = 0
    var cont: Int = 1
    for(i <- pos){
        index = index + cont*i
        cont = cont * 2
    }
    distribution(index)
}

getValueFromDistribution: (distribution: Array[Double], pos: Array[Int])Double


In [7]:
def computeProductOfMessagesForVariableNode(sourceNode: Node): (Double, Double) = {
    var product: (Double, Double) = (1,1)
    for(msg <- sourceNode.messages){
        product = (product._1*msg.value._1, product._2*msg.value._2)
    }
    product
}

computeProductOfMessagesForVariableNode: (sourceNode: Node)(Double, Double)


In [78]:
def checkDestination(messages: List[Message], destinationId: Long): Boolean = {
    var ctrl = true
    for(msg <- messages){
        if(msg.source == destinationId)
            ctrl = false
    }
    ctrl
}

checkDestination: (messages: List[Message], destinationId: Long)Boolean


In [79]:
def computeMessageToFactor(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): Message = {
    if(sourceNode.messages.size == sourceNode.neighbors - 1){
        if(checkDestination(sourceNode.messages, destinationId))
            Message(sourceId, computeProductOfMessagesForVariableNode(sourceNode), destPosInDistribution)
        else
            null
    }else if(sourceNode.messages.size == sourceNode.neighbors){
        val product: (Double, Double) = computeProductOfMessagesForVariableNode(sourceNode)
        if(destinationId == sourceNode.firstDestination)
            null
        else{
            for(msg <- sourceNode.messages){
                if(msg.source == destinationId){
                    Message(sourceId, (product._1/msg.value._1, product._2/msg.value._2), destPosInDistribution)
                }
            }
            null //should not happen
        }
    }else{
        null
    }
}

computeMessageToFactor: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)Message


In [80]:
def computeProductOfMessagesForFactorNode(factorNode: Node, destinationId: Long, states: Array[Int]): Double = {
    var product: Double = 1
    for(msg <- factorNode.messages){
        if(msg.source != destinationId){
            if(states(msg.destPosInDistribution) == 0)
                product = product * msg.value._1
            else
                product = product * msg.value._2
        }
    }
    product
}

computeProductOfMessagesForFactorNode: (factorNode: Node, destinationId: Long, states: Array[Int])Double


In [81]:
def computeMessageToVariableSummation(pos: Int, factorNode: Node, destinationId: Long, states: Array[Int], destPosInDistribution: Int): Double = {
    if(pos == factorNode.neighbors)
        getValueFromDistribution(factorNode.distribution, states)*computeProductOfMessagesForFactorNode(factorNode, destinationId, states)
    else{
        if(pos == destPosInDistribution)
            computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
        else{
            var temp: Double = 0
            states(pos) = 0
            temp = temp + computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
            states(pos) = 1
            temp = temp + computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
            states(pos) = -1
            temp
        }
    }
}

computeMessageToVariableSummation: (pos: Int, factorNode: Node, destinationId: Long, states: Array[Int], destPosInDistribution: Int)Double


In [82]:
def computeMessageToVariable2(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): (Double, Double) = {
    var result: (Double, Double) = (0,0)
    var states = Array.fill[Int](sourceNode.neighbors)(-1)
    states(destPosInDistribution) = 0
    result = (computeMessageToVariableSummation(0, sourceNode, destinationId, states, destPosInDistribution), 0)
    states(destPosInDistribution) = 1
    result = (result._1, computeMessageToVariableSummation(0, sourceNode, destinationId, states, destPosInDistribution))
    result
}
    


computeMessageToVariable2: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)(Double, Double)


In [83]:
def computeMessageToVariable(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): Message = {
    if(sourceNode.messages.size == sourceNode.neighbors - 1){
        if(checkDestination(sourceNode.messages, destinationId))
            Message(sourceId, computeMessageToVariable2(sourceId, sourceNode, destinationId, destPosInDistribution), -1)
        else
            null
    }else if(sourceNode.messages.size == sourceNode.neighbors){
        if(destinationId == sourceNode.firstDestination)
            null
        else{
            Message(sourceId, computeMessageToVariable2(sourceId, sourceNode, destinationId, destPosInDistribution), -1)
        }
        null //should not happen
    }else{
        null
    }
}

computeMessageToVariable: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)Message


In [84]:
import org.apache.spark.AccumulatorParam

object StringAccumulatorParam extends AccumulatorParam[String] {

    def zero(initialValue: String): String = {
        ""
    }

    def addInPlace(s1: String, s2: String): String = {
        s"$s1 $s2"
    }
}

val stringAccum = sc.accumulator("")(StringAccumulatorParam)

defined object StringAccumulatorParam
stringAccum = 


In [85]:
def mergeMsg(msg1: List[Message], msg2: List[Message]): List[Message] = {
    msg1:::msg2
}

mergeMsg: (msg1: List[Message], msg2: List[Message])List[Message]


In [86]:
def vprog(vertexId: VertexId, value: Node, message: List[Message]): Node = {
    if(message.head.source == -1){
        Node(value.typeOfNode, value.neighbors, value.distribution, value.messages, value.firstDestination)
    }
    else{
       
        val newMessages = value.messages:::message
        var firstDestination: Long = -1
        if(value.messages.size == value.neighbors && message.size == 1)
            firstDestination = message.head.source
        Node(value.typeOfNode, value.neighbors, value.distribution, newMessages, firstDestination)
    }
}

vprog: (vertexId: org.apache.spark.graphx.VertexId, value: Node, message: List[Message])Node


In [87]:
//tofix
def sendMsg(triplet: EdgeTriplet[Node, Int]): Iterator[(VertexId, List[Message])] = {
    val sourceId = triplet.srcId
    val sourceNode = triplet.srcAttr
    val destinationId = triplet.dstId
    if(sourceNode.messages.size == 0){
        if(sourceNode.neighbors == 1){
            if(sourceNode.typeOfNode == 0)
                Iterator((destinationId, List(Message(sourceId, (1,1), triplet.attr))))
            else
                Iterator((destinationId, List(Message(sourceId, (getValueFromDistribution(sourceNode.distribution, Array(0)), getValueFromDistribution(sourceNode.distribution, Array(1))), -1))))
        }else{
            Iterator.empty
        }
    }else{
        if(sourceNode.neighbors == 1){
            Iterator.empty
        }else{
            if(sourceNode.typeOfNode == 0){
                val msg = computeMessageToFactor(sourceId, sourceNode, destinationId, triplet.attr)
                //stringAccum += sourceId + " " + destinationId + " " + msg + "\n"
                if(msg == null)
                    Iterator.empty
                else
                    Iterator((destinationId, List(msg)))
            }
            else{
                val msg = computeMessageToVariable(sourceId, sourceNode, destinationId, triplet.attr)
                //stringAccum += sourceId + " " + destinationId + " " + msg + "\n"
                if(msg == null)
                    Iterator.empty
                else
                    Iterator((destinationId, List(msg)))
            }
        }
        
    }
    
  
}

sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[Node,Int])Iterator[(org.apache.spark.graphx.VertexId, List[Message])]


In [88]:
val initialMsg = List(Message(-1, (-1, -1), -1))
val maxIterations = ((variablesNumber+factorsNumber)*2).toInt
val finalGraph = graphWithLeaves.pregel(initialMsg, maxIterations, EdgeDirection.Out)(vprog, sendMsg, mergeMsg)

initialMsg = List(Message(-1,(-1.0,-1.0),-1))
maxIterations = 20
finalGraph = org.apache.spark.graphx.impl.GraphImpl@291a211


org.apache.spark.graphx.impl.GraphImpl@291a211

In [89]:
val temp = finalGraph.vertices.map(x => (x._1, x._2.messages.size, x._2.neighbors))
temp.collect().foreach(println)

(4,0,1)
(0,0,1)
(8,1,2)
(1,1,2)
(9,0,1)
(5,1,2)
(6,3,3)
(2,1,2)
(3,2,3)
(7,0,1)


temp = MapPartitionsRDD[347] at map at <console>:42


MapPartitionsRDD[347] at map at <console>:42

In [90]:
println(stringAccum)